# Annotating Macromolecular Complexes in the Protein Data Bank: Improving the FAIRness of Structure Data

## Disclaimer

This Jupyter notebook is supplementary material for the publication:

**"Annotating Macromolecular Complexes in the Protein Data Bank: Improving the FAIRness of Structure Data"**, Sri A et al., BioRxiv

The code in this notebook reproduces the analysis presented in the publication.

## Background

Macromolecular complexes are crucial functional units in virtually all cellular processes. Their atomic-level understanding is vital to understanding molecular mechanisms and affects applications, such as developing new therapeutics. The Protein Data Bank (PDB) is the central repository for experimentally determined macromolecular structures. However, it can be challenging to find all instances that represent the same assembly in the PDB due to the current PDB annotation practices, which do not include the annotation of assemblies. This study highlights the importance of annotations for macromolecular complexes and the need for more robust methods to identify and classify these complexes across the PDB. We propose a new approach that uses external resources such as the Complex Portal and Gene Ontology to describe assemblies accurately and put them into their biological contexts. We anticipate that the new approach to identifying and classifying complexes will improve the usability and utility of the PDB for researchers in the field of structural biology and drug discovery.

### Primary use case of this Jupyter notebook

This notebook demonstrates how to extract assembly information from the data generated by the weekly-updated complex identifier process of PDBe.
Using this notebook, or by slightly editing the code presented here, it is easy to identify assemblies such as:

* Protein-only assemblies
* Assemblies containing RNA or DNA
* Assemblies with components unmapped to UniProt (e.g. antibodies)

## Importing dependencies

This code imports several modules that are used in the script.

* The `pandas` module is used for data manipulation and analysis
* `utility` is a custom module that contains utility functions used in the script
* `matplotlib` is a plotting library and `pyplot` is a sublibrary within it, often used for creating static, two-dimensional plots.
* `constants` is a custom module that contains constants used in the script. `UNIPROT_PATTERN` and `RFAM_PATTERN` are constants that are defined in the constants module and used in the script.

In [1]:
import csv
from collections import Counter
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re

The code block below contains some variables that store Regex patterns for identifying UniProt/Rfam accessions

In [2]:
# Regex pattern for identifying UniProt accession
UNIPROT_PATTERN = r"([A-N,R-Z][0-9]([A-Z][A-Z, 0-9][A-Z, 0-9][0-9]){1,2})|([O,P,Q][0-9][A-Z, 0-9][A-Z, 0-9][A-Z, 0-9][0-9])(\.\d+)?"
# Regex pattern for identifying Rfam accession
RFAM_PATTERN = r"RF\d{5}"

# This dict contains the definitions of the various unmapped component types
UNMAPPED_COMPONENTS = {
  "all": ("antibody", "Protein_", "RNA_", "DNA_", "DNA/RNA_"),
  "protein": "Protein_",
  "antibody": "antibody_",
  "na": ("RNA_", "DNA_", "DNA/RNA_")
}

## Utility methods

The code block below contains some utility methods used for data analysis

In [3]:
def get_prefered_assembly_str(assembly_str):
 """
 This method returns a string of prefered assemblies
 """
 prefered_assemblies = []
 assemblies = assembly_str.split(",")
 for assembly in assemblies:
    prefered_assembly = prefered_assemblies_dict.get(assembly)
    if prefered_assembly is True:
        prefered_assemblies.append(assembly)
 return ",".join(prefered_assemblies)


def check_for_protein(assembly_string):
  """
  This method checks if the unique assembly contains a protein
  component
  """  
  if re.search(UNIPROT_PATTERN, assembly_string):
    return True
  elif "Protein_" in assembly_string:
    return True
  elif "antibody_" in assembly_string:
    return True
  else:
    return False

def check_for_rna(assembly_string):
  """
  This method checks if the unique assembly contains a RNA
  component
  """  
  if re.search(RFAM_PATTERN, assembly_string):
    return True
  elif "RNA_" in assembly_string:
    return True
  else:
    return False

def check_for_dna(assembly_string):
  """
  This method checks if the unique assembly contains a DNA
  component
  """  
  if "DNA" in assembly_string:
    return True
  else:
    return False

def validate_uniprot(assembly_string, query_type):
  """
  This method checks whether any or all the components in an unique assembly 
  are mapped to UniProt
  """  
  assembly_components = assembly_string.split(",")
  uniprot_matches = [bool(re.search(UNIPROT_PATTERN, component)) for component in assembly_components]
  return query_type(uniprot_matches)

def validate_rfam(assembly_string, query_type):
  """
  This method checks whether any or all the components in an unique assembly
  are mapped to Rfam
  """  
  assembly_components = assembly_string.split(",")
  rfam_matches = [bool(re.search(RFAM_PATTERN, component)) for component in assembly_components]
  return query_type(rfam_matches)

def assembly_composition(valid_protein, valid_RNA, valid_DNA):
  """
  This method returns the assembly polymer composition string
  """
  composition = set()
  if valid_protein:
    composition.add("protein") 
  if valid_RNA:
    composition.add("RNA") 
  if valid_DNA:
    composition.add("DNA")
        
  return ",".join(list(composition))

def get_asymmetrical_assemblies(assembly, sym_operator):
  """
  This method returns a string of assemblies that are
  asymmetrical
  """  
  asymmetrical_assemblies = []
  assembly = assembly.split(",")
  sym_operator = sym_operator.split(",")
  for assembly, sym_operator in zip(assembly, sym_operator):
    if sym_operator == "no-sym":
      asymmetrical_assemblies.append(assembly)
  return ", ".join(asymmetrical_assemblies)

def get_extended_symmetry_operators(assembly, sym_operator):
  """
  This method returns a string of assemblies with their
  symmetry operators
  """   
  assemblies = []
  assembly = assembly.split(",")
  sym_operator = sym_operator.split(",")
  for assembly, sym_operator in zip(assembly, sym_operator):
    assemblies.append(f"{assembly}|{sym_operator}")
  return ", ".join(assemblies)

def most_frequent_symmetry(symmetry_list):
  """
  This method returns the most frequent symmetry operator observed
  for an unique assembly
  """  
  symmetry_list = symmetry_list.split(",")
  occurence_count = Counter(symmetry_list)
  return occurence_count.most_common(1)[0][0]

def get_unique_symmetries(symmetry_list):
  """
  This method returns all the distinct symmetry operators observed
  for an unique assembly joined together as a string
  """  
  result = set(symmetry_list.split(","))
  return ", ".join(list(result))

def get_assembly_type(assembly_string):
  """
  This method returns the assembly type - monomeric, homomeric or 
  heteromeric
  """
  assembly_components = assembly_string.split(",")
  if len(assembly_components) > 1:
    assembly_type = "heteromeric"
  else:
    *_, stoic = assembly_string.split("_")
    if stoic == str(1):
        assembly_type = "monomeric"
    else:
        assembly_type = "homomeric"
  return assembly_type    

def group_UniProt_accessions(data):
  """
  This method returns a dict where the key is an UniProt accession
  while the value is a list of stoichiometry numbers
  """  
  grouped_UNP_accessions = {}
  for elem in data:
    accession, stoichiometry = elem.split("_")
    grouped_UNP_accessions.setdefault(accession, []).append(stoichiometry)
  return grouped_UNP_accessions

def group_subassemblies(data):
  grouped_subassemblies = {}
  grouped_superassemblies = {}
  for elem in data:
    grouped_subassemblies.setdefault(elem[0], []).append(elem[1])
    grouped_superassemblies.setdefault(elem[1], []).append(elem[0])
  return grouped_subassemblies, grouped_superassemblies

def get_sym_op(data):
  """
  This method generates the symmetry operator string for all the assemblies
  of an unique assembly
  """  
  assemblies_list = data.split(",")
  sym_op_list = [SYMMETRY_MAPPING.get(assembly, "no-sym") for assembly in assemblies_list]
  return ",".join(sym_op_list)

def validate_consistent_symmetry(data):
  """
  This method returns True if all assemblies of an unique assembly has the same 
  symmetry operator
  """  
  symmetry_list = data.split(",")
  if len(set(symmetry_list)) == 1:
    return True
  else:
    return False

def count_unique_symmetry(data):
  """
  This method calculates number of unique symmetry observed for an unique
  assembly
  """  
  symmetry_list = data.split(",")
  return len(set(symmetry_list))

def count_assemblies(data):
  """
  This method calculates the number of assemblies for an unique assembly
  """  
  assemblies_list = data.split(",")
  return len(assemblies_list)

def count_unique_pdb(data):
  """
  This method calculates the number of PDB entries for an unique assembly
  """   
  assemblies_list = data.split(",")
  pdb_list = [assembly.split("_")[0] for assembly in assemblies_list]
  return len(set(pdb_list))

def get_sym_variant(ref_symmetry, assemblies):
  """
  This method returns a string of assemblies that do not have the same
  symmetry operator as the rest of assemblies for an unique assembly
  """  
  assemblies = assemblies.split(",")
  variants = []
  for assembly in assemblies:
    _, sym = assembly.split("|")
    if sym != ref_symmetry:
      variants.append(assembly)
  return ", ".join(variants)

def dict_compare(d1, d2):
  """
  This is a general method that compares two dictionaries
  """
  d1_keys = set(d1.keys())
  d2_keys = set(d2.keys())
  shared_keys = d1_keys.intersection(d2_keys)
  added = d1_keys - d2_keys
  removed = d2_keys - d1_keys
  modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
  same = set(o for o in shared_keys if d1[o] == d2[o])
  return added, removed, modified, same

def validate_unmapped(assembly_string, component_type="all"):
  """
  This method checks whether the unique assembly contains unmapped components 
  based on the component_type
  """  
  assembly_components = assembly_string.split(",")
  unmapped_components = [component.startswith(UNMAPPED_COMPONENTS[component_type]) for component in assembly_components]
  return all(unmapped_components)

def get_species_name(assembly_components_string):
  """
  This method returns the species name for an unique assembly based on
  mapped UniProt components
  """  
  assembly_components = assembly_components_string.split(",")
  component_accession = [component.split("_")[0] for component in assembly_components]
  components_names = [UNP_TAX_NAME.get(component) for component in component_accession]
  most_frequent_name = Counter(components_names)
  return most_frequent_name.most_common(1)[0][0]

def get_exp_methods(assemblies_string):
  """
  This method returns a string of experimental-methods used to solve the structures of
  assemblies for an unique assembly
  """  
  exp_method_per_composition = set()
  assemblies = assemblies_string.split(",")
  assemblies = [assembly.split("_")[0] for assembly in assemblies]

  for assembly in assemblies:
      assembly_exp_method = EXP_METHOD.get(assembly)
      if assembly_exp_method == "hybrid":
          exp_method_per_composition.add("Hybrid")
      elif assembly_exp_method == "x-ray":
          exp_method_per_composition.add("X-ray")
      elif assembly_exp_method == "nmr":
          exp_method_per_composition.add("NMR")
      elif assembly_exp_method == "other":
          exp_method_per_composition.add("Other")
      elif assembly_exp_method == "em":
          exp_method_per_composition.add("EM")
      elif assembly_exp_method == "sas":
          exp_method_per_composition.add("SAS")
      
  combined_methods = sorted(list(exp_method_per_composition)) 
  return ",".join(combined_methods) 

def create_prefered_assemblies_string(assemblies, prefered_assemblies):
   """
   This method returns a string of prefered assemblies
   """ 
   assemblies_str_list = []
   for x,y in zip(str(assemblies).split(","), str(prefered_assemblies).split(",")):
      if y == "True":
         assemblies_str_list.append(x)
   return ",".join(assemblies_str_list) 

def check_for_chimeric_chains(assembly_components_str):
   """
   This method checks whether the unique assembly contains chimeric chains
   """ 
   if ":" in assembly_components_str:
      return True
   elif "::" in assembly_components_str:
      return True
   else:
      return False

def format_species_name(name):
    """
    This method formats the species name
    """
    name_components = name.split(" ")
    if len(name_components) == 2:
        name = name
    elif len(name_components) > 2:
        name = " ".join(name_components[:2])
    return name

The code block below contains some variables that store secondary data for analysis

In [4]:
sm = pd.read_csv('https://ftp.ebi.ac.uk/pub/databases/pdbe-kb/complexes/analysis/symmetry_reference.csv')
# Symmetry operator information for protein assemblies in PDB
SYMMETRY_MAPPING = dict(zip(sm.Assembly_ID, sm.Sym_Operator))

ep = pd.read_csv('https://ftp.ebi.ac.uk/pub/databases/pdbe-kb/complexes/analysis/pdb_exp_method.csv')
# Experimental method information for PDB entries
EXP_METHOD = dict(zip(ep.ID, ep.METHOD_CLASS))

sn = pd.read_csv('https://ftp.ebi.ac.uk/pub/databases/pdbe-kb/complexes/analysis/uniprot_tax.csv')
sn['FORMATTED_NAME'] = sn["NAME"].apply(lambda x: format_species_name(x))
# Species name information for Uniprot accessions
UNP_TAX_NAME = dict(zip(sn.ACCESSION, sn.FORMATTED_NAME))

## Pre-Processing

This section contains some pre-processing that we need to do before analysing the data

In [5]:
# Read the CSV file containing the data on assemblies
df = pd.read_csv("https://ftp.ebi.ac.uk/pub/databases/pdbe-kb/complexes/analysis/assemblies_data.csv")
prefered_assemblies_dict = dict(zip(df['ASSEMBLIES'], df['PREFERED_ASSEMBLIES']))

In [6]:
# Group the rows in dataframe based on the assembly string
df = df.astype(str).groupby('ASSEMBLY_STRING', sort=False).agg(lambda x: ','.join(x.unique())).reset_index()
# Drop the existing PREFERED_ASSEMBLIES col in the dataframe
df = df.drop('PREFERED_ASSEMBLIES', axis=1)

In [7]:
# Create a new column that shows the prefered assemblies string for each unique assembly composition
df["PREFERED_ASSEMBLIES"] = df["ASSEMBLIES"].apply(lambda x: get_prefered_assembly_str(x))

In [8]:
# Create a new column that indicates whether each unique assembly composition contains chimeric chains
df["hasChimericChains"] = df["ASSEMBLY_STRING"].apply(lambda x: check_for_chimeric_chains(x))
# Exclude unique assemblies that contain chimeric chains
df = df[df["hasChimericChains"] == False]

## Assembly Classification

We can classify the assemblies into three types:
* monomeric
* homomeric
* heteromeric

A monomeric assembly is defined as an assembly that consists of a single subunit. Homomeric assemblies are formed by repeated units of a single subunit while heteromeric assemblies are composed of multiple distinct subunits. 

In [9]:
df["ASSEMBLY_TYPE"] = df["ASSEMBLY_STRING"].apply(lambda x: get_assembly_type(x))

In [ ]:
# Num of monomeric assemblies
df[df["ASSEMBLY_TYPE"] == "monomeric"].shape[0]

In [ ]:
# Num of homomeric assemblies
df[df["ASSEMBLY_TYPE"] == "homomeric"].shape[0]

In [ ]:
# Num of heteromeric assemblies
df[df["ASSEMBLY_TYPE"] == "heteromeric"].shape[0]

## 1) Assemblies analysis

This section contains some general analyses that were carried out on assemblies across the whole PDB archieve in order to learn their numbers, composition of the assemblies and how many components in the assemblies are mapped to external resources such as [UniProt](https://www.uniprot.org/) for protein entities and [Rfam](https://rfam.org/) for RNA entities

### 1.2) Basic statistics

#### 1.2.1) Unique assemblies

First, we check how many unique assembly compositions are there in the PDB.

In [ ]:
print("There are %i unique assembly compositions in the Protein Data Bank." % len(df))

In [ ]:
# Column to indicate the exp methods used to solved the structures of each unique assembly composition
df["EXP_METHOD"] = df["ASSEMBLIES"].apply(lambda x: get_exp_methods(x))
exp_methods = df["EXP_METHOD"].value_counts().head(20)
exp_methods

In [ ]:
# Column to indicate the species name for each unique assembly composition
df["SPECIES_NAME"] = df["ASSEMBLY_STRING"].apply(lambda x: get_species_name(x))
df["SPECIES_NAME"].value_counts().head(10)

#### 1.2.2) Unique assemblies containing proteins

Next, we look at how many of these unique assemblies contain proteins.

In [ ]:
# Column to indicate the number of assemblies each unique assembly has
df["NUM_ASSEMBLIES"] = df["ASSEMBLIES"].apply(lambda x: count_assemblies(x))

# Column to indicate the number of distinct PDBIDs each unique assembly has
df["NUM_PDB"] = df["ASSEMBLIES"].apply(lambda x: count_unique_pdb(x))

# Column to indicate whether each unique assembly has a protein component
df["hasProteinComponent"] = df["ASSEMBLY_STRING"].apply(lambda x: check_for_protein(x))

df[df["hasProteinComponent"] == True].head()

In [ ]:
print("There are %i unique assemblies containing protein components." % len(df[df["hasProteinComponent"] == True]))

#### 1.2.3) Unique assemblies containing RNAs

We can also count assemblies that contain RNA molecules.

In [ ]:
# Column to indicate whether each unique assembly has a RNA component
df["hasRNAComponent"] = df["ASSEMBLY_STRING"].apply(lambda x: check_for_rna(x))

df[df["hasRNAComponent"] == True]

In [ ]:
print("There %i unique assemblies containing RNA components." % len(df[df["hasRNAComponent"] == True]))

#### 1.2.4) Unique assemblies containing DNAs

Finally, we count the assemblies which contain DNA molecules.

In [ ]:
# Column to indicate whether each unique assembly has a DNA component
df["hasDNAComponent"] = df["ASSEMBLY_STRING"].apply(lambda x: check_for_dna(x))

df[df["hasDNAComponent"] == True].head()

In [ ]:
print("There %i unique assemblies containing DNA components." % len(df[df["hasDNAComponent"] == True]))

### 1.2.5) Summarising what we found so far

Let's summarise how many assemblies fall into specific categories based on their compositions.

In [ ]:
"""
Column to indicate the type of polymer present in each unique assembly. The polymer can either  be protein, 
DNA, or RNA
"""
df["ASSEMBLY_POLYMERS"] = df.apply(lambda row: assembly_composition(row["hasProteinComponent"], row["hasRNAComponent"], row["hasDNAComponent"]), axis=1)

In [ ]:
assemblies_polymer_composition = df["ASSEMBLY_POLYMERS"].value_counts()
assemblies_polymer_composition

Let's visualise the data using a bar chart.

In [ ]:
ax = assemblies_polymer_composition.plot(kind="bar")
ax.set_title("Assemblies polymer composition in PDB")

We can see that the vast majority of assemblies in the PDB are protein assemblies followed by DNA-protein and RNA-protein assemblies respectively.

## 1.3) Focusing on protein assemblies

Next, we will investigate some statistics of protein assemblies.

### 1.3.1) Creating protein-only assembly subset

We start by creating a data frame for protein assemblies.


In [ ]:
# Create a new dataframe to represent protein assemblies
df_protein = df[df["ASSEMBLY_POLYMERS"] == "protein"].copy()
df_protein.head()

### 1.3.2) Assembly components mapped to UniProt accessions

Then, we check if the components in the protein assemblies are all mapped to UniProt accessions. Reasons why a protein may not be mapped to a UniProt accession include:

* Length limitation (sequence too short to map reliably)
* Type limitation (certain classes of proteins, such as antibodies, have no UniProt accessions

In [ ]:
# Column to indicate whether all components in the protein assembly are mapped to UniProt
df_protein["hasAllUNPMapping"] = df_protein["ASSEMBLY_STRING"].apply(lambda x: validate_uniprot(x, all))

In [ ]:
# Number of protein assemblies with all components mapped to UniProt
mapped_protein_assemblies_all = df_protein[df_protein["hasAllUNPMapping"] == True]
mapped_protein_assemblies_all.head()

#### 1.3.2.1) Every component mapped to UniProt accessions

In [ ]:
# Calculate the percentage of protein assemblies with all components mapped to UniProt
print("Across the PDB, %i%% percent of protein assemblies have all of their components mapped to UniProt" % (len(mapped_protein_assemblies_all)/len(df_protein)*100))

1.3.2.2) Any of the components mapped to UniProt accessions

In [ ]:
# Column to indicate whether any one of the components in the protein assembly is mapped to UniProt
df_protein["hasAnyUNPMapping"] = df_protein["ASSEMBLY_STRING"].apply(lambda x: validate_uniprot(x, any))

In [ ]:
# Number of protein assemblies with at least one component mapped to UniProt
mapped_protein_assemblies_any = df_protein[df_protein["hasAnyUNPMapping"] == True]
mapped_protein_assemblies_any.head(10)

In [ ]:
mapped_protein_assemblies_any.shape[0]

In [ ]:
# Calculate the percentage of protein assemblies with at least one component mapped to UniProt
print("Across the PDB archive %i%% of the protein-only assemblies have at least one component mapped to UniProt" % (len(mapped_protein_assemblies_any)/len(df_protein)*100))

## 1.4) Focusing on RNA assemblies

Next, we will investigate some statistics of RNA assemblies.

### 1.4.1) Creating RNA-only assembly subset

We start by creating a data frame for RNA assemblies.

In [ ]:
# Create a new dataframe to represent protein assemblies
df_rna = df[df["ASSEMBLY_POLYMERS"] == "RNA"].copy()
df_rna

### 1.3.2) Assembly components mapped to Rfam accessions
Then, we check if the components in the RNA assemblies are all mapped to Rfam accessions. Reasons why an RNA may not be mapped to a Rfam accession include:

Length limitation (sequence too short to map reliably)
Type limitation (certain types, such as mRNAs, have no Rfam accessions

In [ ]:
# Column to indicate whether all components in the RNA assembly are mapped to Rfam
df_rna["hasAllRfamMapping"] = df_rna["ASSEMBLY_STRING"].apply(lambda x: validate_rfam(x, all))

In [ ]:
# Number of rna assemblies with all components mapped to Rfam
mapped_rna_assemblies_all = df_rna[df_rna["hasAllRfamMapping"] == True]
mapped_rna_assemblies_all.shape[0]

## 2) Variation of stoichiometry in assemblies in the PDB

Structures in the PDB archive are determined to investigate a specific scientific problem. Consequently, a PDB structure does not always represent the biological complex. For example, PDB entry [1abw](https://www.ebi.ac.uk/pdbe/entry/pdb/1abw) contains a human haemoglobin trimer where two copies of the alpha subunit are linked together. This trimer represents a deviation from the biological heterotetrameric state of haemoglobin. In this section of the notebook, we analyse UniProt accessions in which the assemblies exist with different stoichiometries.

In [ ]:
# Create a dataframe to represent monomeric protein assemblies
df_protein_monomeric = df_protein[df_protein["ASSEMBLY_TYPE"] == "monomeric"]
# Create a dataframe to represent homomeric protein assemblies
df_protein_homomeric = df_protein[df_protein["ASSEMBLY_TYPE"] == "homomeric"]
# Create a dataframe to represent heteromeric protein assemblies
df_protein_heteromeric = df_protein[df_protein["ASSEMBLY_TYPE"] == "heteromeric"]

# Calculate the percentage of monomeric protein assemblies
percentage_monomeric_assemblies = (len(df_protein_monomeric)/len(df_protein))*100
# Calculate the percentage of homomeric protein assemblies
percentage_homomeric_assemblies = (len(df_protein_homomeric)/len(df_protein))*100
# Calculate the percentage of heteromeric protein assemblies
percentage_heteromeric_assemblies = (len(df_protein_heteromeric)/len(df_protein))*100

In [ ]:
print("Out of all the protein assemblies in the PDB, %i%% are monomeric, %i%% are homomeric and %i%% are heteromeric." % (
    percentage_monomeric_assemblies,
    percentage_homomeric_assemblies,
    percentage_heteromeric_assemblies
))

### 2.1) Homomeric protein assemblies

Next, we will investigate the homomeric protein assemblies.

In [ ]:
# Create a dataframe to represent homomeric protein assemblies with all components mapped to UniProt
df_protein_homomeric_mapped = df_protein_homomeric[(df_protein_homomeric["hasAllUNPMapping"] == True)]
df_protein_homomeric_mapped

Show the top 10 species of the homomeric assemblies

In [ ]:
homomeric_species_count = df_protein_homomeric_mapped["SPECIES_NAME"].value_counts().head(10)
homomeric_species_count

In [ ]:
hx = homomeric_species_count.plot(kind="bar")
hx.set_title("Top 10 species from which homomeric assemblies are solved")

In [ ]:
# Calculate the percentage of homomeric protein assemblies that have all components mapped to UniProt
percentage_homomeric_assemblies_mapped = (len(df_protein_homomeric_mapped)/len(df_protein_homomeric))*100

In [ ]:
print("%i%% of the homomeric protein assemblies have all their components mapped to UniProt accession." % percentage_homomeric_assemblies_mapped)

We then create a list of assembly composition strings, and group them by UniProt accessions.

In [ ]:
# Create a list of UniProt accessions with stoichiometry in the homomeric assemblies
homomeric_assembly_strings = df_protein_homomeric_mapped["ASSEMBLY_STRING"].tolist()

In [ ]:
# Group the UniProt accessions in homomeric assemblies
homomeric_grouped_UNP_accessions = group_UniProt_accessions(homomeric_assembly_strings)
print ("There are %i groups." % len(homomeric_grouped_UNP_accessions))

Next, we keep only those groups that has more than one assembly states.

In [ ]:
# Only keep UniProt accessions that display 2 or more stoichiometries
homomeric_UNP_accessions_with_muliple_stoic = {k: sorted([int(x) for x in grp]) for k, grp in homomeric_grouped_UNP_accessions.items() if len(grp) > 1}
# Number of UniProt accessions with multiple stoichiometry numbers
homomeric_UNP_accession_stoic_count = len(homomeric_UNP_accessions_with_muliple_stoic)

In [ ]:
print("There are %i UniProt accessions that exist in different stoichiometries in homomeric assemblies" % homomeric_UNP_accession_stoic_count)

In [ ]:
# Count the stoichiometry number for UniProt accessions with two or more stoichiometries
homomeric_UNP_accessions_with_multiple_stoic_count = [(k, len(grp)) for k, grp in homomeric_UNP_accessions_with_muliple_stoic.items()]

In [ ]:
# Sort the count of stoichiometry number in descending order
sorted_homomeric_UNP_accessions_with_multiple_stoic_count = sorted(homomeric_UNP_accessions_with_multiple_stoic_count, key=lambda x: x[1], reverse=True)

In [ ]:
# Show the top 10 UniProt accessions with highest stoichiometry number count
sorted_homomeric_UNP_accessions_with_multiple_stoic_count[:12]

### 2.2) Heteromeric protein assemblies

Now, we will investigate the heteromeric protein assemblies.

In [ ]:
# Create a dataframe to represent heteromeric protein assemblies with all components mapped to UniProt
df_protein_heteromeric_mapped = df_protein_heteromeric[(df_protein_heteromeric["hasAllUNPMapping"] == True)]
df_protein_heteromeric_mapped.head()

In [ ]:
# Calculate the percentage of heteromeric protein assemblies that have all components mapped to UniProt
percentage_heteromeric_assemblies_mapped = (len(df_protein_heteromeric_mapped)/len(df_protein_heteromeric))*100

In [ ]:
print("%i%% of the heteromeric protein assemblies have all their components mapped to UniProt accession." % percentage_heteromeric_assemblies_mapped)

In [ ]:
# Create a list of UniProt accessions with stoichiometry in the heteromeric assemblies
heteromeric_assembly_strings = df_protein_heteromeric_mapped["ASSEMBLY_STRING"].tolist()
heteromeric_assembly_components = [assembly.split(",") for assembly in heteromeric_assembly_strings]
heteromeric_assembly_components = [item for sublist in heteromeric_assembly_components for item in sublist] 
heteromeric_assembly_components = list(set(heteromeric_assembly_components))

In [ ]:
# Group the UniProt accessions in the heteromeric assemblies
heteromeric_grouped_UNP_accessions = group_UniProt_accessions(heteromeric_assembly_components)
print ("There are %i groups." % len(heteromeric_grouped_UNP_accessions))

Next, we keep only those groups with more than one assembly states.

In [ ]:
# Only keep UniProt accessions that display 2 or more stoichiometry number
heteromeric_UNP_accessions_with_muliple_stoic = {k: sorted([int(x) for x in grp]) for k, grp in heteromeric_grouped_UNP_accessions.items() if len(grp) > 1}
# Number of UniProt accessions with multiple stoichiometry numbers
heteromeric_UNP_accession_stoic_count = len(heteromeric_UNP_accessions_with_muliple_stoic)

In [ ]:
print("There are %i UniProt accessions that exist in different stoichiometries in heteromeric assemblies" % heteromeric_UNP_accession_stoic_count)

In [ ]:
# Count the stoichiometry number for UniProt accessions with two or more stoichiometries
heteromeric_UNP_accessions_with_multiple_stoic_count = [(k, len(grp)) for k, grp in heteromeric_UNP_accessions_with_muliple_stoic.items()]

In [ ]:
# Sort the count of stoichiometry number in descending order
sorted_heteromeric_UNP_accessions_with_multiple_stoic_count = sorted(heteromeric_UNP_accessions_with_multiple_stoic_count, key=lambda x: x[1], reverse=True)

In [ ]:
# Show the top 10 UniProt accessions with highest stoichiometry number count
sorted_heteromeric_UNP_accessions_with_multiple_stoic_count[:12]

We now compare the UniProt accessions that are used by homomeric and heteromeric assemblies

In [ ]:
added, removed, modified, same = dict_compare(homomeric_UNP_accessions_with_muliple_stoic, heteromeric_UNP_accessions_with_muliple_stoic)

In [ ]:
print("There are %i UniProt accessions unique for homomeric assemblies" % len(added))

In [ ]:
print("There are %i UniProt accessions unique for heteromeric assemblies" % len(removed))

In [ ]:
print("There are %i UniProt accessions that are common to both homomeric and heteromeric assemblies with the same assembly states" % len(same))

In [ ]:
print("There are %i UniProt accessions that are common to both homomeric and heteromeric assemblies with varying assembly states" % len(modified))

## 3) Sub-assembly analysis

A biological assembly can consist of multiple components where each component can have varying stoichiometry. This is best illustrated by the ribosome which is the molecular machine responsible for protein synthesis in every living cell. The ribosome is made up of two ribosomal subunits (small and large ribosomal subunits) and can bind multiple tRNAs (1-3 tRNAs), mRNA, and diverse protein factors. The binding of tRNAs and the various protein factors can induce the ribosome to undergo large-scale conformational changes which affects how the ribosome functions. As such, the ability to identify the sub- and super-assemblies of a biological assembly is key in identifying the contribution of each component to the function of the whole assembly and the relationships that exist between the components.

We define sub- and super-assembly using the composition and stoichiometry data. We defined a sub-assembly as an assembly where all components map to UniProt accessions and where these components are also present in another assembly. For example, PDB entry [5jdo](https://pdbe.org/5jdo) contains haemoglobin (UniProt [P68871](https://pdbe-kb.org/proteins/P68871) and UniProt [P69905](https://pdbe-kb.org/proteins/P69905)) bound to the haptoglobin-haemoglobin receptor (UniProt [G0UVW6](https://pdbe-kb.org/proteins/G0UVW6)). A sub-assembly of this assembly is haemoglobin. We defined a super-assembly as an assembly that has all the same elements of another assembly plus additional members. Again, all members must map to a UniProt accession. In the example above,  PDB entry [5jdo](https://pdbe.org/5jdo is a super-assembly of haemoglobin.

### 3.1) Reading in the data

In [ ]:
df_subassemblies = pd.read_csv("https://ftp.ebi.ac.uk/pub/databases/pdbe-kb/complexes/analysis/subassemblies_data.csv")
df_subassemblies.head()

In [ ]:
print("There are %i unique sub-assemblies, and %i unique super-assemblies in the PDB." % (
    df_subassemblies["SUBASSEMBLY_ID"].nunique(),
    df_subassemblies["SUPERASSEMBLY_ID"].nunique()
))

### 3.2) Matching sub- and super-assemblies

We can match sub- and super-assemblies.

In [ ]:
subassembly_list = df_subassemblies["SUBASSEMBLY_ID"].tolist()
superassembly_list = df_subassemblies["SUPERASSEMBLY_ID"].tolist()
subassembly_pairs = list(zip(subassembly_list, superassembly_list))

In [ ]:
subassembly_pairs[:10]

In [ ]:
grouped_subassemblies, grouped_superassemblies = group_subassemblies(subassembly_pairs)

In [ ]:
# Assemblies having at least two super-assemblies
assemblies_with_muliple_superassembly = {k: grp for k, grp in grouped_subassemblies.items() if len(grp) > 1}
print("Only %i assemblies have 2 or more super-assemblies" % len(assemblies_with_muliple_superassembly))

In [ ]:
# Assemblies having at least two super-assemblies
assemblies_with_muliple_subassembly = {k: grp for k, grp in grouped_superassemblies.items() if len(grp) > 1}
print("Only %i assemblies have 2 or more sub-assemblies" % len(assemblies_with_muliple_subassembly))

### 3.3) Finding the assemblies in the PDB which have the highest number of super-assemblies

In [ ]:
# Count the number of superassemblies
superassemblies_count = [(k, len(grp)) for k, grp in assemblies_with_muliple_superassembly.items()]

# Sort the count of superassemblies in descending order
sorted_superassemblies_count = sorted(superassemblies_count, key=lambda x: x[1], reverse=True)

# Show the top 10 entries with the highest number of superassemblies
sorted_superassemblies_count[:10]

# Get the PDB Complex ID for the top 10 entries
top_ten_entries_ids = [x[0] for x in sorted_superassemblies_count[:10]]

top_ten_entries_ids

# Show the assemblies data for the top 10 entries
df[df["PDB_COMPLEX_ID"].isin(top_ten_entries_ids)]

# 4) Naming of Assemblies

Providing a good name for a complex in the PDB poses a significant challenge. Complexes with a singular component can get the name of this component, regardless of the stoichiometry of the component. Examples of this would be Lysozyme, Insulin or Ferritin. In assemblies where the singular component is either DNA or RNA, and no mapping to external resources is present, the name is given as DNA or RNA, respectively. In the case of proteins, we can use the protein name the depositor provided if no external mapping is available. If the component maps to a UniProt accession, then we can use the protein name from UniProt. This approach allows us to automatically name some of the multi-component assemblies, including proteins bound to nucleic acids.


In [ ]:
# Num of assemblies that are complexes
pdb_complexes = df[df["ASSEMBLY_TYPE"] != "monomeric"]
pdb_complexes

## 4.1) Count the number of assemblies that have a name


In [ ]:
sum_named_assemblies = (df['ASSEMBLY_NAME'] != 'nan').sum()
percentage_named_assemblies = sum_named_assemblies/df.shape[0]*100

In [ ]:
print("Sum of assemblies with name: {}".format(sum_named_assemblies))

In [ ]:
print("Percentage of assemblies with name: {:.1f}%".format(percentage_named_assemblies))

## 4.2) Count the number of names generated from each naming category

In [ ]:
df["NAME_SOURCE"].value_counts()

We can see that majority of the names are derived from UniProt

## 4.3) Count the total number of names derived from UniProt

### 4.31) Breakdown the number of names derived from each UniProt category

In [ ]:
up = df[df["NAME_SOURCE"].str.contains("UniProt", na=False)]
print(up.groupby("NAME_SOURCE").size().sort_values(ascending=False))

In [ ]:
# Num of unique assemblies 
up.shape[0]

### 4.32) Count the number of PDB entries that have names derived from UniProt

In [ ]:
df.loc[df["NAME_SOURCE"].str.contains("UniProt", na=False), 'NUM_PDB'].sum()

## 4.4) Count the total number of names derived from Complex Portal

In [ ]:
# Num of unique assemblies 
df["NAME_SOURCE"].str.contains("complex portal").sum()

### 4.41) Breakdown the number of names derived from each Complex Portal category

In [ ]:
assemblies_with_ComplexPortal_names = df[df["NAME_SOURCE"].str.contains("complex portal", na=False)]
print(assemblies_with_ComplexPortal_names.groupby("NAME_SOURCE").size().sort_values(ascending=False))

### 4.42) Count the number of PDB entries that have names derived from Complex Portal

In [ ]:
df.loc[df["NAME_SOURCE"].str.contains("complex portal", na=False), 'NUM_PDB'].sum()

## 4.5) Count the number of unique assemblies & PDB entries that are named as heterodimer

In [ ]:
condition = df["NAME_SOURCE"] == "heterodimer"
hd = df[condition]
# Num of unique assemblies
hd.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.6) Count the number of unique assemblies & PDB entries that are named as ribosomes

In [ ]:
condition = df["NAME_SOURCE"].str.contains("ribosome")
ribosome = df[condition]
# Num of unique assemblies
ribosome.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.7) Count the number of unique assemblies & PDB entries that have names derived from Gene Ontology

In [ ]:
condition = df["NAME_SOURCE"] == "Gene Ontology"
go = df[condition]
# Num of unique assemblies
go.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

In [ ]:
go["ASSEMBLY_NAME"].value_counts()

## 4.8) Count the number of unique assemblies & PDB entries that have manually curated names

In [ ]:
condition = df["NAME_SOURCE"] == "PDBe curated"
mc = df[condition]
# Num of unique assemblies
mc.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.9) Count the number of unique assemblies & PDB entries that have general nucleic acid names

In [ ]:
condition = df["NAME_SOURCE"] == "general nucleic acid name"
gn = df[condition]
# Num of unique assemblies
gn.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.10) Count the number of unique assemblies & PDB entries that have common entity names

In [ ]:
condition = df["NAME_SOURCE"] == "common name from entity names"
en = df[condition]
# Num of unique assemblies
en.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.11) Count the number of unique assemblies & PDB entries that have all antibodies names

In [ ]:
condition = df["NAME_SOURCE"] == "all_antibodies"
ab = df[condition]
# Num of unique assemblies
ab.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.12) Count the number of unique assemblies & PDB entries that have all unmapped protein names

In [ ]:
condition = df["NAME_SOURCE"] == "all_unmapped_proteins_without_antibodies"
aup = df[condition]
# Num of unique assemblies
aup.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.13) Count the number of PDB entries that have Rfam names

In [ ]:
condition = df["NAME_SOURCE"].str.contains("RNA name from Rfam")
rf = df[condition]
# Num of unique assemblies
rf.shape[0]

In [ ]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

# 5) Symmetry analysis

PDB assemblies represent different forms that certain complex molecules, like proteins, can take. For example, a protein called haemoglobin is known to exist in two different states, one called the relaxed state, or R state, and the other called the tense state, or T state. These two states are structurally distinct from each other, meaning they have different shapes and forms. In order to gain a better understanding of these different shapes and forms, we used a computer program called [AnAnaS](https://team.inria.fr/nano-d/software/ananas/), which stands for "Analyse and Annotate Structures". This program is used to study the symmetry of different PDB assemblies, which can provide insights into the conformational heterogeneity of these complex molecules.

## 5.1) Load symmetry data

First we add several columns to the protein assembly data set:

* Symmetry operators
* A flag for consistent symmetry across various instances of the same assembly
* The number of distinct symmetry operator each unique assembly composition has

In [ ]:
# Add a column to indicate the symmetry operators of the assemblies for each unique assembly composition
df_protein["SYMMETRY_OPS"] = df_protein["PREFERED_ASSEMBLIES"].apply(lambda x: get_sym_op(x))

# Add a column to indicate whether all assemblies for each unique assembly composition have consistent symmetry
df_protein["CONSISTENT_SYMMETRY"] = df_protein["SYMMETRY_OPS"].apply(lambda x: validate_consistent_symmetry(x))

# Add a column to indicate the number of distinct symmetry operators each unique assembly composition has
df_protein["SYM_NUM"] = df_protein["SYMMETRY_OPS"].apply(lambda x: count_unique_symmetry(x))

# Add a column to show all the unique symmetries for each unique assembly composition
df_protein["ALL_SYM"] = df_protein["SYMMETRY_OPS"].apply(lambda x: get_unique_symmetries(x))

# Add a column to show the most frequent symmetry for each unique assembly composition
df_protein["PRIMARY_SYM"] = df_protein["SYMMETRY_OPS"].apply(lambda x: most_frequent_symmetry(x))

## 5.2) Symmetry statistics

In [ ]:
df_crystal = df['EXP_METHOD'] == 'X-ray'
df_consistent = df_protein[(df_protein['CONSISTENT_SYMMETRY'] == True) & (df_protein['PRIMARY_SYM'] != "no-sym") & df_crystal]

In [ ]:
print(f"There are {df_consistent.shape[0]} unique assemblies that have a symmetry")

In [ ]:
cyclic_assemblies = df_consistent[df_consistent['PRIMARY_SYM'].str.contains("c")]
dihedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'].str.contains("d")]
tetrahedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'] == "t"]
octahedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'] == "o"]
icosahedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'] == "i"]

In [ ]:
print(f"There are {cyclic_assemblies.shape[0]} cyclic assemblies, {dihedral_assemblies.shape[0]} dihedral assemblies, {tetrahedral_assemblies.shape[0]} tetrahedral assemblies, {octahedral_assemblies.shape[0]} octahedral assemblies and finally, {icosahedral_assemblies.shape[0]} icosahedral assemblies")

In [ ]:
# Show all the cyclic assemblies variants
cyclic_assemblies_variants = cyclic_assemblies['PRIMARY_SYM'].value_counts()
cyclic_assemblies_variants

In [ ]:
cx = cyclic_assemblies_variants.plot(kind="bar")
cx.spines['top'].set_visible(False)
cx.spines['right'].set_visible(False)
cx.set_yscale('log')
cx.set_xticklabels(cx.get_xticklabels(), rotation=0)

In [ ]:
# Show all the dihedral assemblies variants
dihedral_assemblies_variants = dihedral_assemblies['PRIMARY_SYM'].value_counts()
dihedral_assemblies_variants

In [ ]:
dx = dihedral_assemblies_variants.plot(kind="bar")
# dx.set_title("Dihedral (Dn) symmetry groups detected")
dx.spines['top'].set_visible(False)
dx.spines['right'].set_visible(False)
dx.set_yscale('log')
dx.set_xticklabels(dx.get_xticklabels(), rotation=0)

## 5.3) Listing assemblies with inconsistent symmetries

We can list assemblies where the symmetry data is inconsistent between different instances of the same assembly. This can highlight potential annotation errors, or genuinely interesting assemblies.

In [ ]:
df_inconsistent_symmetry = df_protein[df_protein['CONSISTENT_SYMMETRY'] == False]
# Number of unique assemblies containing members with inconsistent symmetry
df_inconsistent_symmetry.shape[0]

We can also investigate the data by explicitly excluding asymmetric C1 instance.

In [ ]:
# Assemblies with inconsistent symmetry operators (excluding asymmetric C1)
df_protein[(df_protein["CONSISTENT_SYMMETRY"] == False) & (~df_protein["SYMMETRY_OPS"].str.contains("no-sym"))]

## Conclusion

This work provides an archive-wide analysis of assemblies in the PDB archive and demonstrates the power of mapping individual components of assemblies to external databases. Our new approach allows the otherwise complex aggregation of assemblies. With the help of mappings to external databases, it also became possible to name over 80% of assemblies in the PDB, representing over 90% of PDB entries. These names allow users of PDBe to find complexes that previously could only be found by relying on the title of the PDB entry or listing all the complex components. Searching by title or listing components is likely to lead to incomplete and inconsistent search results due to the heterogeneity of assemblies in the PDB archive. We are making the complexes name list and mapping data files available to the community to aid in analysing and identifying macromolecular complexes.

## Authors

* Sri Appasamy
* Mihaly Varadi

### Last updated March 2023